
---

### ❓ **1. What happens if multiple users run queries on a Small warehouse?**

---

### ✅ **Correct Answer:**

If the number of concurrent queries exceeds the capacity of the Small warehouse, the **excess queries will queue** until others finish.

---

### 🧠 Explanation:

* A Small warehouse has **limited compute resources** — let’s say (hypothetically) it can handle 8 queries at a time.
* If 12 users run queries simultaneously:

  * 8 run immediately.
  * 4 are **queued** until resources free up.

Snowflake handles this queuing **automatically**. No queries fail — just delayed.

---

### ⚠️ Common Misconception:

> "Snowflake auto-scales the warehouse size automatically to handle this."

**Incorrect.** Snowflake will **NOT** resize your warehouse automatically.
Only **multi-cluster warehouses** can scale horizontally to handle more concurrent queries.

---

### 📘 Real-World Analogy:

Imagine a restaurant kitchen with 2 chefs. They can prepare 2 dishes at a time.
If 5 orders come in at once, only 2 dishes start cooking — the other 3 wait.
That’s exactly what a Small warehouse does.

---

### ✅ Fix: Use a **Multi-Cluster Warehouse** with auto-scaling for concurrency problems.

---

---

### ❓ **2. Can a warehouse resize itself automatically?**

---

### ✅ **Correct Answer:**

**No**, a warehouse **cannot resize itself automatically** from Small → Medium → Large.

---

### 🧠 Explanation:

* Snowflake doesn’t support **automatic vertical scaling**.
* If you want to resize a warehouse:

  * You must manually change its size.
  * You must suspend it first, then change the size, then resume.

```sql
ALTER WAREHOUSE my_wh SUSPEND;
ALTER WAREHOUSE my_wh SET WAREHOUSE_SIZE = 'MEDIUM';
ALTER WAREHOUSE my_wh RESUME;
```

This causes a short downtime.

---

### ⚠️ Common Misconception:

> “Snowflake is cloud-native, so it must auto-resize compute like AWS Lambda or Kubernetes.”

**Wrong assumption.** Snowflake warehouses are **user-defined compute allocations**.

---

### ✅ Fix: For dynamic scaling, use **Multi-Cluster** (horizontal scaling), not size resizing.

---

---

### ❓ **3. How do multi-cluster warehouses help reduce queuing?**

---

### ✅ **Correct Answer:**

Multi-cluster warehouses **add more clusters automatically** when concurrency increases, thus **reducing query queuing**.

---

### 🧠 Explanation:

* You define a **MIN\_CLUSTER\_COUNT** and **MAX\_CLUSTER\_COUNT**.
* Snowflake starts with `min` number of clusters.
* When concurrent queries spike and the main cluster is saturated, Snowflake:

  * Adds more **clusters (same size)** up to `max`.
* Each new cluster can handle another batch of queries **in parallel**.

---

### 📘 Real Example:

```sql
CREATE WAREHOUSE dashboard_wh
  WAREHOUSE_SIZE = 'SMALL'
  MIN_CLUSTER_COUNT = 1
  MAX_CLUSTER_COUNT = 3;
```

If 20 queries hit at once:

* Cluster 1 takes the first batch.
* Cluster 2 and 3 spin up automatically and take the rest — **no queuing**!

---

### ✅ Bonus: When traffic reduces, Snowflake **auto-spins down** the extra clusters to save cost.

---

---

### ❓ **4. What’s the difference between vertical and horizontal scaling in Snowflake?**

---

### ✅ **Correct Answer:**

| Scaling Type       | What It Means                                                   | Snowflake Support             |
| ------------------ | --------------------------------------------------------------- | ----------------------------- |
| Vertical Scaling   | Increase compute power of **one cluster** (e.g., Small → Large) | ✅ Manual Only                 |
| Horizontal Scaling | Add **more clusters** of same size to handle more queries       | ✅ Automatic via Multi-Cluster |

---

### 🧠 Explanation:

* **Vertical scaling** = Increase size of 1 engine = More CPU/Memory
  Useful for **long/heavy queries**.

* **Horizontal scaling** = Add more engines = More parallel query execution
  Useful for **concurrent users/dashboards**.

---

### ⚠️ Critical Point:

Snowflake **doesn’t auto-scale vertically**.
But it **does auto-scale horizontally** with multi-cluster config.

---

### 📘 Analogy:

* **Vertical** = Upgrading a food truck’s kitchen to be more powerful.
* **Horizontal** = Hiring 3 identical food trucks to serve more customers.

---

---

### ❓ **5. How can you save costs with high concurrency workloads?**

---

### ✅ **Correct Answer:**

Use a **Small warehouse** in **Multi-Cluster Mode** with:

* `AUTO_SUSPEND` enabled
* `AUTO_RESUME` enabled
* Reasonable `MAX_CLUSTER_COUNT` based on observed concurrency

---

### 🧠 Explanation:

* Using a **larger warehouse** (like 2X-Large) for high concurrency is wasteful if most queries are small.
* Instead, use multiple **Small clusters**, scaling out only when needed.

This way:

* You **only pay for extra clusters when traffic spikes**
* Each cluster **auto-suspends** when idle
* You control max limits with `MAX_CLUSTER_COUNT`

---

### ✅ Example:

```sql
CREATE WAREHOUSE bi_dashboard_wh
  WAREHOUSE_SIZE = 'SMALL'
  AUTO_SUSPEND = 60
  AUTO_RESUME = TRUE
  MIN_CLUSTER_COUNT = 1
  MAX_CLUSTER_COUNT = 4;
```

---

### 📘 Real Case:

Your dashboard team accesses Snowflake all day, but with fluctuating traffic.

* Early morning: 1 cluster runs.
* 10 AM: 4 clusters spin up due to heavy access.
* Noon: Traffic drops → 3 clusters suspend.

You **pay only for what was used**, while keeping dashboards responsive.

---

## ✅ Recap: Key Answers at a Glance

| Question                             | Summary Answer                              |
| ------------------------------------ | ------------------------------------------- |
| 1. Multiple users on Small warehouse | Excess queries queue                        |
| 2. Auto-resize warehouse?            | ❌ No — manual only                          |
| 3. Multi-cluster reduces queuing?    | ✅ Yes — adds clusters on demand             |
| 4. Vertical vs Horizontal Scaling    | Vertical = size; Horizontal = cluster count |
| 5. Cost saving for concurrency       | Use multi-cluster SMALL + auto-suspend      |

---

